In [ ]:
# structure nsl data as objects
from nsl_class import NSL

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime

# parse nslarchive pages given the directory
def parse_nsl_pages_and_extract_urls(data_dir):
    files = sorted(os.listdir(data_dir))
    files = [f for f in files if '.html' in f]
    
    # Save all links
    nsls = []
    for f in files:
        print('processing ' + f)
        # read the raw html
        with open("{}/{}".format(data_dir, f)) as fin:
            html = fin.read()
        # parse html
        soup = BeautifulSoup(html)
        
        # find the table that list nsl letters
        table = soup.find_all('table')
        # not all pages contain nsl pdfs
        if len(table) == 0:
            continue
        else:
            # find the table that contains nsl pdfs
            table = table[0]
        
        # record data for each nsl letter
        nsl_file_number = None
        nsl_file_url = None
        nsl_date = None
        nsl_release_date = None
        nsl_release_url = None
        nsl_company = None
        nsl_comment = None
        idx = 0
        links_to_nsl_file = []
        for tr in table.find_all('tr'):
            # skip first tr
            if 'How many?' in tr.text:
                continue
            for td in tr.find_all('td'):
                if idx % 6 == 0:
                    nsl_release_date = datetime.strptime(td.text, "%Y-%m-%d")
                if idx % 6 == 1:
                    company = td.text.strip()
                if idx % 6 == 2:
                    pass
                if idx % 6 == 3:
                    print('here')
                    for a in td.find_all('a'):
                        links_to_nsl_file.append(a['href'])
                    print(links_to_nsl_file)
                    links_to_nsl_file = []
                        # nsls.append(NSL(nsl_issue_date, nsl_file_number_only, company="Apple", year=nsl_issue_date.year, release_date=nsl_release_date, link_to_nsl_file=nsl_file_url))
                if idx % 6 == 5:
                    nsl_comment = td.text.strip()
                idx += 1
                
    print('total urls: {}'.format(len(nsls)))
    return nsls

def download_file_at_url(url):
    print(url)
    response = requests.get(url)
    with open("{}/{}".format(DATA_DIR, url.split("/")[-1]), 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {url.split('/')[-1]}")
    time.sleep(1)
    

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time

# Directory where we store all html pages that contain links to nsl
DATA_DIR = "../../data/raw/nslarchive_nsl_pages"
# Get urls for all apple pdfs
nsls = parse_nsl_pages_and_extract_urls(DATA_DIR)

# # Store parsed data in CSV file
# APPLE_CSV_FILE = "../../data/extracted/nsl_letters_apple.csv"

# with open(APPLE_CSV_FILE, "w") as csv_file:
#     csv_file.write("issue date, release date, company, file number, url, comment\n")
#     for nsl in nsls:
#         csv_file.write(nsl.export_as_csv_line() + "\n")
        
# # Download data
# DATA_DIR = "../../data/raw/nsl/apple"
# for nsl in nsls:
#     download_file_at_url(nsl.link_to_nsl_file)